In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

import sys

sys.path.append('..')

from flod.classifiers.bsvclassifier import BSVClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from scipy.stats import randint, uniform

In [ ]:
def plot_contour(X, clf, colors):
    # Creates a grid to fit all the data
    gx = np.linspace(min(X[:,0])-.2, max(X[:,0])+.2, 50)
    gy = np.linspace(min(X[:,1])-.2, max(X[:,1])+.2, 50)
    gX, gY = np.meshgrid(gx, gy)
    
    # Evaluates the radious of each point in the 2 dimensional space
    radiuses = [clf._compute_r(np.array([x, y])) for x, y in zip(np.ravel(gX), np.ravel(gY))]
    zs = np.array(radiuses)
    gZ = zs.reshape(gX.shape)
    membership_contour = plt.contour(gX, gY, gZ, levels=(clf.radius_, ), colors='blue')
    plt.clabel(membership_contour, inline=1)

    plt.scatter(X[:,0], X[:,1], c=colors, alpha=.2)
    plt.show()

In [ ]:
def print_report(clf, dataset):
    mistakes = sum(clf.predict(X))
    print(f'c: {clf.c}, q: {clf.q}')

    sv_count = 0
    bsv_count = 0
    for b in clf.betas_:
        if not np.isclose(b, 0) and not np.isclose(b, clf.c):
                    sv_count+=1
        if np.isclose(b, clf.c):
                    bsv_count+=1

    print(f'Support Vectors are {sv_count}, over {len(dataset)} candidates')
    print(f'Bounded Support Vectors are {bsv_count}, over {len(dataset)} candidates')
    print(f'Mistakes {mistakes}/{len(dataset)} = {mistakes/len(dataset)*100}%')

In [ ]:
def count_zeros(y, y_pred, **kwargs):
    return len(y)-sum(y_pred)

scoring = {
    'zeros_scorer': make_scorer(count_zeros)
}

# FLOD goes federated over synthetic data

In this notebook we want to answer the following question:
"Can the server build a sphere containing all the data since the dataset is distributed to all the clients and unkown to the server?"

There is a fundamental issue in this test:
there are no outliers, so the naive 100% correct sphere would be the simplest and biggest one that can fit the space.

## 1. Single client scenario

We generate a synthetic dataset. 
All the points have label 0, since 1 means the points is an outlier.

With multiple centers, a low std is harder than a large one.

I have tested with centers from 1 to 5 and with std up to 2.
I also tried to raise the number of samples, but soon we hit the limit of how many points gurobi can handle

In [ ]:
X, y = make_blobs(n_samples=1000, centers=5, n_features=2, cluster_std=1.0)
y = [0] * len(X)

In [ ]:
X

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

Let's see if we can create a sphere containing all the points with our implementation of the classifier.

In [ ]:
params = {
    'q': uniform(0.0001, 20),
    'c': uniform()    
}
clf = RandomizedSearchCV(BSVClassifier(), params, cv=2, n_jobs=-1, scoring=scoring, refit='zeros_scorer', return_train_score=False, n_iter=15)
clf.fit(X, y)

Theoretically, in the following plot there should be only one color, the one representing the points classified to 0.

In [ ]:
best_clf = clf.best_estimator_

plot_contour(X, best_clf, best_clf.predict(X))

In [ ]:
print_report(best_clf, X)

Some times even this baseline does around 20% of mistakes.

## 2. Multiple client scenario

Let's split randomly the data generated before among k clients

In [ ]:
clients = 5
points_per_client = int(len(X) / clients)

In [ ]:
k = [X[i*points_per_client: (i+1)*points_per_client] for i in range(clients)]

colors = [int(i/points_per_client) for i in range(len(X))]

Let's plot again the previous plot, but this time the colors represent the client assignment

In [ ]:
best_clf = clf.best_estimator_
plot_contour(X, best_clf, colors)

### Federated primitives

#### 1. Init server model

In [ ]:
def init_server_model():
    return {
        'q': 1,
        'C': 1,
        'betas': np.empty(shape=(0, 2)),
        'xs': np.empty(shape=(0, 2))
    }

#### 2. Client update

In [ ]:
def client_compute_update(global_model, client_data):
    # Concat points from server and from client
    X = np.concatenate((global_model['xs'], client_data))
    # Init the classifier with q and C from server
    clf = BSVClassifier(q=global_model['q'], c=global_model['C'])
    # Train locally
    clf.fit(X, [0]*len(X))
    # Select only the positive betas related from client_data
    client_betas = clf.betas_[len(global_model['xs']):]
    assert(len(client_betas) == len(client_data))
    
    for i, t in enumerate(zip(client_betas, client_data)):
        b, x = t
        if not np.isclose(b, 0):
            yield x
    

#### 3. Server combines client updates

In [ ]:
def global_combine(global_model, client_updates):
    # Concatenates server points and clients candidate points.
    X = np.concatenate((global_model['xs'], *client_updates))
    
    # Performs model selection over this new dataset
    # Cross validation is low because I want to fit exactly the data I have got
    clf = RandomizedSearchCV(BSVClassifier(), params, cv=2, n_jobs=-1, scoring=scoring, refit='zeros_scorer', return_train_score=False, n_iter=15)
    clf.fit(X, [0] * len(X))
    
    # Filter and keep only the support vectors
    xs = []
    betas = []
    for i, t in enumerate(zip(clf.best_estimator_.betas_, X)):
        b, x = t
        if not np.isclose(b, 0):
            xs.append(x)
            betas.append(b)
            
    return {
        'q': clf.best_estimator_.q,
        'C': clf.best_estimator_.c,
        'betas': betas,
        'xs': xs
    }, clf.best_estimator_

## Federated Learning simulation with one pass on data

In [ ]:
import threading

number_of_rounds = 20

server_model = init_server_model()

# Metrics
debug_sk_models = []
debug_models = []
mistakes = []

client_updates = []

points_per_round = int(points_per_client / number_of_rounds)

def client_worker(client, round_ix):
    dataset = k[client]
    # Pick a data slice
    dataset = dataset[points_per_round*round_ix : points_per_round*(round_ix+1),:]
    update = client_compute_update(server_model, dataset)
    update = np.array(list(update))
    if update.size > 0:
        client_updates.append(update)

for r in tqdm(range(number_of_rounds)):
    client_updates = []
    #threads = []
    for client in range(clients):
        client_worker(client, r)
        #t = threading.Thread(target=client_worker, args=(client, r))
        #threads.append(t)
        #t.start()
        
    #for t in threads:
        #t.join()
        
    server_model, debug_model = global_combine(server_model, client_updates)
    mistakes.append(sum(debug_model.predict(X)))
    debug_models.append(server_model)
    debug_sk_models.append(debug_model)

In [ ]:
plot_contour(X, debug_model, colors)

In [ ]:
print_report(debug_model, X)

**How does the mistakes ratio change during optimization?**

At each round we collect data about the mistakes of the model over the whole dataset

In [ ]:
plt.plot([len(s['betas']) for s in debug_models], label='support vectors')
plt.plot(mistakes, label='mistakes')
plt.legend()
plt.show()

In [ ]:
print(f'Average mistakes: {np.average(mistakes)}')
print(f'Std mistakes: {np.std(mistakes)}')

Usually the standard deviation is very high, probabily keeping track of the best model so far is a good idea.

How can we evaluate which model is the best in so far in production since we have limited view about the dataset?

### How does the best classifier found during the process work?

In [ ]:
best_mistakes = min(mistakes)
best_mistakes_index = mistakes.index(best_mistakes)

print(f'Best mistakes {best_mistakes}/{len(X)} = {best_mistakes/len(X)*100}% at round {best_mistakes_index}/{number_of_rounds}')

In [ ]:
plot_contour(X, debug_sk_models[best_mistakes_index], colors)
print_report(debug_sk_models[best_mistakes_index], X)

## Federated Learning simulation with multiple pass on data

Does seing the data multiple times reduce mistakes? It's a sort of overfit
So at line 17, instead of taking the following slice of data, we sample the client's data randomly at each iteration.

It can happen that the same data is seen multiple times.

In [ ]:
number_of_rounds_sampling = 70

server_model_sampling = init_server_model()

# Metrics
debug_sk_models_sampling = []
debug_models_sampling = []
mistakes_sampling = []

client_updates = []

points_per_round_sampling = points_per_round

def client_worker_sampling(client, round_ix):
    dataset = k[client]
    # Randomly sample the client data
    random_batch = np.random.choice(dataset.shape[0], points_per_round_sampling) 
    dataset = dataset[random_batch]
    
    update = client_compute_update(server_model_sampling, dataset)
    update = np.array(list(update))
    if update.size > 0:
        client_updates.append(update)

for r in tqdm(range(number_of_rounds_sampling)):
    client_updates = []
    #threads = []
    for client in range(clients):
        client_worker_sampling(client, r)
        #t = threading.Thread(target=client_worker_sampling, args=(client, r))
        #threads.append(t)
        #t.start()
        
    #for t in threads:
        #t.join()
        
    server_model_sampling, debug_model = global_combine(server_model, client_updates)
    mistakes_sampling.append(sum(debug_model.predict(X)))
    debug_models_sampling.append(server_model)
    debug_sk_models_sampling.append(debug_model)

The pink line shows the number of iterations at which the "single pass on data" simulation ended. (previous experiment)

In [ ]:
print(f'Mistakes {mistakes_sampling[-1]}/{len(X)} = {mistakes_sampling[-1]/len(X)*100}%')

plt.plot([len(s['betas']) for s in debug_models_sampling], label='support vectors')
plt.plot(mistakes_sampling, label='mistakes')
plt.axvline(number_of_rounds, alpha=.4, color='purple')
plt.legend()
plt.show()

In [ ]:
print(f'Average mistakes: {np.average(mistakes_sampling)}')
print(f'Std mistakes: {np.std(mistakes_sampling)}')

In [ ]:
plot_contour(X, debug_sk_models_sampling[-1], colors)
print_report(debug_sk_models_sampling[-1], X)

### How does the best classifier found during the process work?

In [ ]:
best_mistakes = min(mistakes_sampling)
best_mistakes_index = mistakes_sampling.index(best_mistakes)

print(f'Best mistakes {best_mistakes}/{len(X)} = {best_mistakes/len(X)*100}% at round {best_mistakes_index}/{number_of_rounds_sampling}')

In [ ]:
plot_contour(X, debug_sk_models_sampling[best_mistakes_index], colors)
print_report(debug_sk_models_sampling[best_mistakes_index], X)